## やりたいこと
設定値の辞書化  
→メイン変数が乱立しすぎて非常に見通しが悪い&名前空間が混乱しやすいので設定値は一つの辞書にまとめる  
(セッティングモードも辞書だけでやり取りできるようになるため実装が容易になるメリットもある)

In [7]:
import sys
import os
import configparser
import json
import pyaudio

os.chdir("C:\\Users\\amane\\OneDrive\\python\\proto2021\\bin")
if os.path.exists(".\\init_test"):
    pass
else:
    os.mkdir(".\\init_test")
    print("test directory created.")

os.chdir(".\\init_test")
base_dir = os.getcwd() #os.path.dirname(__file__) の代理
print(base_dir)


C:\Users\amane\OneDrive\python\proto2021\bin\init_test


---
デフォルト値も辞書で定義する

In [8]:
#初期値の定義
cfg = {
    "br" : 8,
    "sr" : 22050,
    "r_wait" : 5,
    "r_len" : 3,
    "a_idx" : None,
    "disp_spg" : False,

    "axis_freq" : None,
    "axis_tme" : None,
    "aug_amt" : 512,

    "exp_ver" : 0.999,
    "model_pca" : "PCAmodel",       #PCAモデルのファイル名
    "model_keras" : "keras",        #kerasのモデルフォルダ
    "model_tf_keras" : "tf_keras",  #tf.kerasのモデルフォルダ

    "data_dir" : "waves",           #集音データの保管フォルダ
    "model_dir" : "models",         #モデルファイルの保管フォルダ
    "log_dir" : "logs",             #tensorboardのログファイル保管フォルダ

}

---

`Init_boot`クラスの中でも設定値は変数化せずに辞書と直接やり取りさせる (`base_dir`だけはグローバル変数との比較のために変数化)  
将来的に設定値が増えても初期辞書定義とゲッター/セッターへの追記のみで済む (`return`や関数呼び出しの変数順序を気にしなくてよくなる)

In [9]:
class init_boot():

    def __init__(self,cfg):
        self.cfg = cfg
        self.base_dir = base_dir

        #パス関連の初期値を相対パス化
        self.cfg["data_dir"] = os.path.join(".\\",self.cfg["data_dir"])
        self.cfg["model_dir"] = os.path.join(".\\",self.cfg["model_dir"])
        self.cfg["log_dir"] = os.path.join(".\\",self.cfg["log_dir"])
        self.cfg["model_keras"] = os.path.join(
            self.cfg["model_dir"],
            self.cfg["model_keras"]
        )
        self.cfg["model_tf_keras"] = os.path.join(
            self.cfg["model_dir"],
            self.cfg["model_tf_keras"]
        )

        #PCAモデルファイル初期値生成
        self.cfg["model_pca"] = os.path.join(
            "proto2021_094" #os.path.splitext(os.path.basename(__file__))[0]
            + "_" + self.cfg["model_pca"] + ".dat"
        )

        #インスタンスの生成
        self.pa = pyaudio.PyAudio()
        self.cp = configparser.ConfigParser()


        """フォルダの生成"""
    def  elem_gen_folder(self,target_dir):
        if os.path.exists(target_dir):
            pass
        else:
            os.mkdir(target_dir)
            print("Created a directory:{0}".format(target_dir))

    #辞書からiniファイルの内容をセットする
    def elem_set_ini(self):
        self.cp["General"] = {
            "base_dir" : self.base_dir,
            "data_dir" : self.cfg["data_dir"],
            "model_dir" : self.cfg["model_dir"],
            "log_dir" : self.cfg["log_dir"]
        }

        self.cp["Rec_param"] = {
            "br" : self.cfg["br"],
            "sr" : self.cfg["sr"],
            "r_wait" : self.cfg["r_wait"],
            "r_len" : self.cfg["r_len"],
            "disp_spg" : self.cfg["disp_spg"]
        }

        if self.cfg["a_idx"] == None:
            self.cp["Rec_param"]["a_idx"] = ""
        else:
            self.cp["Rec_param"]["a_idx"] = str(self.cfg["a_idx"])

        self.cp["Train_param"] = {
            "aug_amt" : self.cfg["aug_amt"],
            "exp_ver" : self.cfg["exp_ver"],
            "model_pca" : self.cfg["model_pca"],
            "model_keras" : self.cfg["model_keras"],
            "model_tf_keras" : self.cfg["model_tf_keras"]
        }

        if self.cfg["axis_freq"] == None:
            self.cp["Train_param"]["axis_freq"] = ""
        else:
            self.cp["Train_param"]["axis_freq"] = self.cfg["axis_freq"]

        if self.cfg["axis_tme"] == None:
            self.cp["Train_param"]["axis_tme"] = ""
        else:
            self.cp["Train_param"]["axis_tme"] = self.cfg["axis_tme"]

    """iniファイルへの保存"""
    def elem_save_ini(self):
        with open(
            "proto2021_094" + ".ini","w") as cfgfile:
            self.cp.write(cfgfile)
        print ("Saved setting parameters.")

    """iniファイルのロード"""
    def elem_load_ini(self):
        self.cp.read(
            os.path.join(
                "./","proto2021_094" + ".ini"
            )
        )
        print ("Loded initial setting parameters.")

    #iniファイルから設定値を読み出し辞書に格納
    def elem_get_ini(self):

        #読み出し
        self.cfg["data_dir"] = self.cp.get("General","data_dir")
        self.cfg["model_dir"] = self.cp.get("General","model_dir")
        self.cfg["log_dir"] = self.cp.get("General","log_dir")

        self.cfg["br"] = self.cp.getint("Rec_param","br")
        self.cfg["sr"] = self.cp.getint("Rec_param","sr")
        self.cfg["r_wait"] = self.cp.getint("Rec_param","r_wait")
        self.cfg["r_len"] = self.cp.getint("Rec_param","r_len")
        self.cfg["disp_spg"] = self.cp.getboolean("Rec_param","disp_spg")

        if self.cp.get("Rec_param","a_idx") == "":
            self.cfg["a_idx"] = None
        else:
            self.cfg["a_idx"] = int(self.cp.getint("Rec_param","a_idx"))

        self.cfg["aug_amt"] = self.cp.getint("Train_param","aug_amt")
        self.cfg["exp_ver"] = self.cp.getfloat("Train_param","exp_ver")
        self.cfg["model_pca"] = self.cp.get("Train_param","model_pca")
        self.cfg["model_keras"] = self.cp.get("Train_param","model_keras")
        self.cfg["model_tf_keras"] = self.cp.get("Train_param","model_tf_keras")

        if self.cp.get("Train_param","axis_freq") == "":
            self.cfg["axis_freq"] = None
        else:
            self.cfg["axis_freq"] = json.loads(self.cp.get("Train_param","axis_freq"))

        if self.cp.get("Train_param","axis_tme") == "":
            self.cfg["axis_tme"] = None
        else:
            self.cfg["axis_tme"] = json.loads(self.cp.get("Train_param","axis_tme"))
        
    """オーディオインデックスの定義"""
    def elem_a_idx(self):
        print("***List of available audio devices:***")
        for i in range(self.pa.get_device_count()):
            print(i,self.pa.get_device_info_by_index(i).get("name"),sep = " - ")
        x = int(input("Select Audio device Index No."))
        print("***Selected audio device #{0}.***".format(x))
        return x

    #起動処理
    def proc_boot(self):
        #iniファイルの有無判定
        if os.path.exists(os.path.join("./","proto2021_094" + ".ini")):
            #iniファイルが存在すればロード
            self.elem_load_ini()
            #前回起動時とカレントディレクトリを比較
            if self.cp.get("General","base_dir") != self.base_dir:
                #カレントディレクトリが違う場合初期起動と同じ処理をする
                print("The directory has changed since the last boot. Reinitialize the settings.")
                self.cfg["a_idx"] = self.elem_a_idx()
                self.elem_set_ini()
                self.elem_save_ini()
            else:
                #カレントディレクトリが変わっていなければ設定値をセット
                self.elem_get_ini()
        else:
            #iniファイルが存在しない場合、オーディオインデックスを定義しセーブ
            self.cfg["a_idx"] = self.elem_a_idx()
            self.elem_set_ini()
            self.elem_save_ini()

        #フォルダの生成
        self.elem_gen_folder(self.cfg["data_dir"])
        self.elem_gen_folder(os.path.join(self.cfg["data_dir"],"ok"))
        self.elem_gen_folder(os.path.join(self.cfg["data_dir"],"ng"))
        self.elem_gen_folder(os.path.join(self.cfg["data_dir"],"valid"))
        self.elem_gen_folder(self.cfg["model_dir"])
        self.elem_gen_folder(self.cfg["log_dir"])

        return self.cfg


In [4]:
Boot = init_boot(cfg)
cfg = Boot.proc_boot()

***List of available audio devices:***
0 - Microsoft Sound Mapper - Input
1 - デスクトップ マイク (Microsoft® LifeCam 
2 - マイク配列 (Realtek(R) Audio)
3 - Microsoft Sound Mapper - Output
4 - SHARP HDMI (インテル(R) ディスプレイ用オーディ
5 - Speakers (Realtek HD Audio output)
6 - マイク配列 1 (Realtek HD Audio Mic input with SST)
7 - マイク配列 2 (Realtek HD Audio Mic input with SST)
8 - マイク配列 3 (Realtek HD Audio Mic input with SST)
9 - Output (インテル(R) ディスプレイ用オーディオ - 出力 1)
10 - デスクトップ マイク (Microsoft® LifeCam HD-3000)
11 - ヘッドホン ()
12 - ヘッドホン ()
13 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(pianist_dannaのAirPods Pro))
14 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(pianist_dannaのAirPods Pro))
15 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(JAMBOX by Jawbone))
16 - ヘッドセット (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(JAMBOX by Jawbone))
17 - Output (@System32\drivers\bthhfenum.sys,#4;%1 Hands-Free HF Audio%0
;(Xperia 10))
18 

In [5]:
cfg

{'br': 8,
 'sr': 22050,
 'r_wait': 5,
 'r_len': 3,
 'a_idx': 1,
 'disp_spg': False,
 'axis_freq': None,
 'axis_tme': None,
 'aug_amt': 512,
 'exp_ver': 0.999,
 'model_pca': 'proto2021_094_PCAmodel.dat',
 'model_keras': '.\\models\\keras',
 'model_tf_keras': '.\\models\\tf_keras',
 'data_dir': '.\\waves',
 'model_dir': '.\\models',
 'log_dir': '.\\logs'}

初期設定の保存に成功

---

読み出しテスト

In [6]:
del cfg,Boot,base_dir

この間にセル1～3を再実行

In [10]:
Boot = init_boot(cfg)
cfg = Boot.proc_boot()

Loded initial setting parameters.


In [11]:
cfg

{'br': 8,
 'sr': 22050,
 'r_wait': 5,
 'r_len': 3,
 'a_idx': 1,
 'disp_spg': False,
 'axis_freq': None,
 'axis_tme': None,
 'aug_amt': 512,
 'exp_ver': 0.999,
 'model_pca': 'proto2021_094_PCAmodel.dat',
 'model_keras': '.\\models\\keras',
 'model_tf_keras': '.\\models\\tf_keras',
 'data_dir': '.\\waves',
 'model_dir': '.\\models',
 'log_dir': '.\\logs'}

---

## わかったこと
- 起動処理の呼び出し時の引数が辞書一つになるので記述が非常にすっきりする
- クラス内も辞書のまま直接処理したほうが、変数への代入や辞書への書き戻し処理が必要ないためコード記述量が減り可読性も上がる
- 辞書の特性上、キーにスペルミスがあると「ミスったままのキーと値」が追加されてしまう上、コーディング上IDEも効かないので注意が必要
- 辞書は仕様上キーの位置は定まっていないが、表示錠は最初に定義した/追加した順で表示される